# Text Representation
文本表示

## Preparation for Google Colab

In [ ]:
import os
from google.colab import drive

#  挂载 google 云盘
drive.mount("/content/drive")

print(os.getcwd())  # /content

# print(os.listdir("/content/drive/MyDrive/"))

# print(os.listdir("/content/drive/MyDrive/Colab Notebooks"))

# if os.getcwd() != "/content/drive/MyDrive":
#     os.chdir("/content/drive/MyDrive")

# print(os.getcwd())

In [ ]:
# 提前将 requirements.txt 放在 google 云盘上
!pip install -r /content/drive/MyDrive/requirements.txt

In [3]:
subdir = "ch07a"
work_path = "/content/drive/MyDrive/" + subdir
if not os.path.exists(work_path):
    os.mkdir(work_path)
os.chdir(work_path)
print(os.getcwd())

/content/drive/MyDrive/ch07a


In [4]:
!apt-get install tree && tree -a "./"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tree is already the newest version (2.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
./

0 directories, 0 files


## datasets

In [5]:
import evaluate

metric = evaluate.load("glue", "stsb")
metric.compute(predictions=[1, 2, 3], references=[5, 2, 2])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'pearson': -0.8660254037844388, 'spearmanr': -0.8660254037844387}

In [6]:
from datasets import load_dataset
import evaluate

stsb = load_dataset("glue", "stsb")
stsb_metric = evaluate.load("glue", "stsb")

In [7]:
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer

use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
distilroberta = SentenceTransformer("stsb-distilroberta-base-v2")

In [8]:
import tensorflow as tf
import math


def use_sts_benchmark(batch):
    sts_encode1 = tf.nn.l2_normalize(use_model(tf.constant(batch["sentence1"])), axis=1)
    sts_encode2 = tf.nn.l2_normalize(use_model(tf.constant(batch["sentence2"])), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    return scores

In [9]:
def roberta_sts_benchmark(batch):
    sts_encode1 = tf.nn.l2_normalize(distilroberta.encode(batch["sentence1"]), axis=1)
    sts_encode2 = tf.nn.l2_normalize(distilroberta.encode(batch["sentence2"]), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    return scores

In [10]:
use_results = use_sts_benchmark(stsb["validation"])
distilroberta_results = roberta_sts_benchmark(stsb["validation"])

In [11]:
references = [item["label"] for item in stsb["validation"]]

In [12]:
results = {
    "USE": stsb_metric.compute(predictions=use_results, references=references),
    "DistillRoberta": stsb_metric.compute(
        predictions=distilroberta_results, references=references
    ),
}

In [13]:
import pandas as pd

pd.DataFrame(results)

,USE,DistillRoberta
pearson,0.810301,0.888461
spearmanr,0.808917,0.889246
